In [217]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import utils
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd


In [218]:
players_info_list = utils.get_dataframes_csv_file('../scraping/csv/players_infos')
players_stats_list = utils.get_dataframes_csv_file('../scraping/csv/players_stats')

In [219]:
players_stats_list_summer_league = []
players_stats_list_playoffs = []
players_stats_list_regular_season = []
players_stats_list_preseason = []

for df in players_stats_list:
    year = df['Title'].iloc[0].split(' ')[1]

    if year == '2023-2024':
        continue

    season_type = df['Title'].iloc[0].split(' ')[2]
    
    # Ajouter les données au dictionnaire approprié
    if season_type == 'Summer_League':
        players_stats_list_summer_league.append(df)
    elif season_type == 'Playoffs':
        players_stats_list_playoffs.append(df)
    elif season_type == 'Regular_Season':
        players_stats_list_regular_season.append(df)
    elif season_type == 'Preseason':
        players_stats_list_preseason.append(df)

In [220]:
players_df_list = []

for stats_df in players_stats_list:
    players_list = []

    for index, row in stats_df.iterrows():
        # Obtention du titre du DataFrame de statistiques
        title = row['Title']

        # Obtention de l'année et du type de saison à partir du titre
        year_season = title.split(' ')[1]

        # Obtention du nom du joueur à partir de la colonne 'Player'
        player_name = row['Player']

        # Recherche du DataFrame correspondant dans players_info_list
        info_df = None
        for info_df in players_info_list:
            info_title = info_df['Title'].iloc[0]
            if year_season in info_title:
                break

        if info_df is not None:
            # Recherche de la ligne correspondante au joueur dans le DataFrame info_df
            player_info = info_df[info_df['Player'] == player_name]

            if not player_info.empty:
                # Création d'une liste contenant les informations du joueur
                player_data = [
                    player_name,
                    player_info['Pos'].iloc[0],
                    player_info['HT'].iloc[0],
                    player_info['WT'].iloc[0],
                    player_info['Age'].iloc[0],
                    player_info['Teams'].iloc[0],
                    row['GP'],
                    player_info['YOS'].iloc[0],
                    player_info['Pre-Draft Team'].iloc[0],
                    player_info['Draft Status'].iloc[0],
                    player_info['Nationality'].iloc[0],
                    row['Team'],
                    row['MIN'],
                    row['PTS'],
                    row['FGM'],
                    row['FGA'],
                    row['FG%'],
                    row['3PM'],
                    row['3PA'],
                    row['3P%'],
                    row['FTM'],
                    row['FTA'],
                    row['FT%'],
                    row['ORB'],
                    row['DRB'],
                    row['REB'],
                    row['AST'],
                    row['STL'],
                    row['BLK'],
                    row['TOV'],
                    row['PF'],
                    row['Title']
                ]

                players_list.append(player_data)
            else:
                # Si aucune correspondance n'est trouvée, ajouter des zéros pour chaque colonne
                player_data = [
                    player_name,
                    0,
                    0,
                    0,
                    0,
                    0,
                    row['GP'],
                    0,
                    0,
                    0,
                    0,
                    row['Team'],
                    row['MIN'],
                    row['PTS'],
                    row['FGM'],
                    row['FGA'],
                    row['FG%'],
                    row['3PM'],
                    row['3PA'],
                    row['3P%'],
                    row['FTM'],
                    row['FTA'],
                    row['FT%'],
                    row['ORB'],
                    row['DRB'],
                    row['REB'],
                    row['AST'],
                    row['STL'],
                    row['BLK'],
                    row['TOV'],
                    row['PF'],
                    row['Title']
                ]

                players_list.append(player_data)

    # Création du DataFrame à partir de la liste des données des joueurs
    columns = ['Player', 'Pos', 'HT', 'WT', 'Age', 'Teams', 'GP', 'YOS', 'Pre-Draft Team', 'Draft Status',
            'Nationality', 'Team', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM',
            'FTA', 'FT%', 'ORB', 'DRB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF','Title']

    players_df = pd.DataFrame(players_list, columns=columns)

    players_df_list.append(players_df)

In [221]:
# Fonction de conversion de pieds et pouces en mètres
def convert_to_meters(ht : str):
    if ht == '' or ht == 0:
        return None

    feet, inches = ht.split('-')
    if(inches == ''):
        if(feet == ''):
            return None
        inches = '0'

    total_inches = int(feet) * 12 + int(inches)
    meters = total_inches * 0.0254
    return meters

# Appliquer la fonction de conversion à la colonne "HT" pour obtenir la taille en mètres a tout nos données
for df in players_df_list:
    df['HT'] = df['HT'].apply(convert_to_meters)

In [222]:
# Rendre toutes les données numérique 
for df in players_df_list:
    # non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns

    for column in ['MIN', 'PTS', 'FGA', 'REB']:
        df[column] = df[column].apply(lambda x: None if x == '-' else float(x.replace(',', '')) if isinstance(x, str) else x)

In [266]:
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output

# app = dash.Dash(__name__)

# dataframes = players_df_list

# # Définir les statistiques disponibles
# stats_options = [
#     {'label': 'Points per Player', 'value': 'PTS'},
#     {'label': 'Games Played per Player', 'value': 'GP'},
#     {'label': 'Minutes Played per Player', 'value': 'MIN'},
#     {'label': 'Field Goals Made per Player', 'value': 'FGM'},
#     {'label': 'Field Goals Attempted per Player', 'value': 'FGA'},
#     {'label': 'Field Goal Percentage per Player', 'value': 'FG%'},
#     {'label': '3-Point Field Goals Made per Player', 'value': '3PM'},
#     {'label': '3-Point Field Goals Attempted per Player', 'value': '3PA'},
#     {'label': '3-Point Field Goal Percentage per Player', 'value': '3P%'},
#     {'label': 'Free Throws Made per Player', 'value': 'FTM'},
#     {'label': 'Free Throws Attempted per Player', 'value': 'FTA'},
#     {'label': 'Free Throw Percentage per Player', 'value': 'FT%'},
#     {'label': 'Offensive Rebounds per Player', 'value': 'ORB'},
#     {'label': 'Defensive Rebounds per Player', 'value': 'DRB'},
#     {'label': 'Total Rebounds per Player', 'value': 'REB'},
#     {'label': 'Assists per Player', 'value': 'AST'},
#     {'label': 'Steals per Player', 'value': 'STL'},
#     {'label': 'Blocks per Player', 'value': 'BLK'},
#     {'label': 'Turnovers per Player', 'value': 'TOV'},
#     {'label': 'Personal Fouls per Player', 'value': 'PF'}
# ]

# app.layout = html.Div(
#     children=[

#         html.H1("Basketball Player Stats Dashboard"),

#         # Affichage des statistiques de tout les joueurs par saison
#         html.H2("Static Visualizations"),
#         dcc.Dropdown(
#             id="season-dropdown",
#             options=[
#                 {"label": f"Season {dataframes[i]['Title'].iloc[0]}", "value": i} for i in range(len(dataframes))
#             ],
#             value=0
#         ),
#         dcc.RadioItems(
#             id="stats-radio",
#             options=stats_options,
#             value=stats_options[0]['value']
#         ),
#         html.Div(id="static-visualizations"),

#     ]
# )

# @app.callback(
#     Output("static-visualizations", "children"),
#     [Input("season-dropdown", "value"), Input("stats-radio", "value")]
# )
# def update_static_visualizations(season_id, selected_stat):
#     selected_df = dataframes[season_id]

#     figure = {
#         "data": [
#             {
#                 "x": selected_df["Player"],
#                 "y": selected_df[selected_stat],
#                 "type": "bar",
#                 "name": selected_stat
#             }
#         ],
#         "layout": {
#             "title": f"{selected_stat} per Player"
#         }
#     }
#     graph = dcc.Graph(figure=figure)

#     return graph

# @app.callback(
#     Output("dynamic-visualization", "children"),
#     [Input("player-dropdown", "value")]
# )
# def update_dynamic_visualization(player_id):
#     selected_df = dataframes[0]  # Commencer par le dataframe de la première saison 
#     figure = {
#         "data": [
#             {
#                 "x": selected_df["AST"],
#                 "y": selected_df["REB"],
#                 "mode": "markers",
#                 "marker": {
#                     "size": selected_df["PTS"] / 10,
#                     "color": selected_df["Nationality"],
#                     "colorscale": "Viridis",
#                     "showscale": True
#                 },
#                 "text": selected_df["Player"]
#             }
#         ],
#         "layout": {
#             "title": f"Assists vs. Rebounds for {selected_df['Player'].iloc[0]}"
#         }
#     }
#     return dcc.Graph(figure=figure)

In [385]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

dataframes = players_df_list

stats_options = [
    {'label': 'Points per Player', 'value': 'PTS'},
    {'label': 'Games Played per Player', 'value': 'GP'},
    {'label': 'Minutes Played per Player', 'value': 'MIN'},
    {'label': 'Field Goals Made per Player', 'value': 'FGM'},
    {'label': 'Field Goals Attempted per Player', 'value': 'FGA'},
    {'label': 'Field Goal Percentage per Player', 'value': 'FG%'},
    {'label': '3-Point Field Goals Made per Player', 'value': '3PM'},
    {'label': '3-Point Field Goals Attempted per Player', 'value': '3PA'},
    {'label': '3-Point Field Goal Percentage per Player', 'value': '3P%'},
    {'label': 'Free Throws Made per Player', 'value': 'FTM'},
    {'label': 'Free Throws Attempted per Player', 'value': 'FTA'},
    {'label': 'Free Throw Percentage per Player', 'value': 'FT%'},
    {'label': 'Offensive Rebounds per Player', 'value': 'ORB'},
    {'label': 'Defensive Rebounds per Player', 'value': 'DRB'},
    {'label': 'Total Rebounds per Player', 'value': 'REB'},
    {'label': 'Assists per Player', 'value': 'AST'},
    {'label': 'Steals per Player', 'value': 'STL'},
    {'label': 'Blocks per Player', 'value': 'BLK'},
    {'label': 'Turnovers per Player', 'value': 'TOV'},
    {'label': 'Personal Fouls per Player', 'value': 'PF'}
]

stats_options_extension = [
    {'label': 'Team', 'value': 'Teams'},
    {'label': 'Position', 'value': 'Pos'},
    {'label': 'Height', 'value': 'HT'},
    {'label': 'Weight', 'value': 'WT'},
    {'label': 'Position', 'value': 'Age'},
    {'label': 'Nationality', 'value': 'Nationality'},
] + stats_options

app.layout = html.Div(
    children=[
        html.H1("Basketball Player Stats Dashboard"),

        # Affichage des statistiques de tout les joueurs par saison
        html.H2("Static Visualization"),
        dcc.Dropdown(
            id="season-dropdown",
            options=[
                {"label": f"Season {dataframes[i]['Title'].iloc[0]}", "value": i} for i in range(len(dataframes))
            ],
            value=0
        ),
        dcc.RadioItems(
            id="stats-radio",
            options=stats_options,
            value=stats_options[0]['value']
        ),
        html.Div(id="static-visualizations"),


        # Affichage des statistiques joueur par joueur
        html.H2("Dynamic Visualization"),
        dcc.Dropdown(
            id="player-dropdown",
            placeholder="Select a player"
        ),

        # TODO : affiche les statistique du joueur selectionné dans "player-dropdown" 
        html.Div(id="player-stats")  # Div pour afficher les statistiques du joueur sélectionné
        # html.H2("---------------"),
    
    ]
)

@app.callback(
    Output("static-visualizations", "children"),
    [Input("season-dropdown", "value"), Input("stats-radio", "value")]
)
def update_static_visualizations(season_id, selected_stat):
    selected_df = dataframes[season_id]

    figure = {
        "data": [
            {
                "x": selected_df["Player"],
                "y": selected_df[selected_stat],
                "type": "bar",
                "name": selected_stat
            }
        ],
        "layout": {
            "title": f"{selected_stat} per Player"
        }
    }
    graph = dcc.Graph(figure=figure)

    return graph

@app.callback(
    Output("player-dropdown", "options"),
    [Input("season-dropdown", "value")]
)
def update_player_dropdown(season_id):
    selected_df = dataframes[season_id]
    players = selected_df["Player"].unique()
    player_options = [{"label": player, "value": player} for player in players]
    return player_options



@app.callback(
    Output("player-stats", "children"),
    [Input("player-dropdown", "value"), Input("season-dropdown", "value")]
)
def update_player_stats(selected_player, season_id):
    selected_df = dataframes[season_id]
    player_stats = selected_df[selected_df["Player"] == selected_player]

    if len(player_stats) == 0:
        return html.Div("No data available for the selected player.")

    stat_labels = [stat_option["label"] for stat_option in stats_options_extension]

    table_rows = [
        html.Tr([
            html.Td(stat_label),
            html.Td(player_stats[stat_value].iloc[0])
        ]) for stat_label, stat_value in zip(stat_labels, [stat_option["value"] for stat_option in stats_options_extension])
   ]  # Fermeture de la liste des lignes de la table

    table = html.Table([
        html.Thead(html.Tr([html.Th("Stat"), html.Th("Value")])),  # En-tête de la table
        html.Tbody(table_rows)  # Corps de la table avec les lignes
    ])

    return table

In [386]:
import socket

if __name__ == "__main__":
    
    my_port = 8050  # Le port par défaut utilisé par Dash
    # app.run_server(debug=True, dev_tools_hot_reload=False)
    app.run_server("0.0.0.0", port = my_port, debug = True, dev_tools_hot_reload=False)
    ip = socket.gethostbyname(socket.gethostname())
    url = f"http://{ip}:{port}/"
    print(f"Dash app running at: {url}")

#http://192.168.137.39:8050


ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_03991e26-7342-48b4-a2b3-3e648acfdc1d (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000175F1904690>: Failed to establish a new connection: [WinError 10049] L’adresse demandée n’est pas valide dans son contexte'))